In [1]:
# coding:utf-8
# 各種import
import gym  #gymのインポート
from gym import wrappers  #gymの画像保存
import numpy as np #天下のnumpy
import time

In [2]:
# -------- Q関数を離散化して定義する関数　------------
# 観測した状態を離散値にデジタル変換する
def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]


# 各値を離散値に変換
def digitize_state(observation):
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [
        np.digitize(cart_pos, bins=bins(-2.4, 2.4, num_dizitized)),
        np.digitize(cart_v, bins=bins(-3.0, 3.0, num_dizitized)),
        np.digitize(pole_angle, bins=bins(-0.5, 0.5, num_dizitized)),
        np.digitize(pole_v, bins=bins(-2.0, 2.0, num_dizitized))
    ]
    return sum([x * (num_dizitized**i) for i, x in enumerate(digitized)])

In [3]:
# ---------- 行動a(t)を求める関数 ---------------------------
def get_action(next_state, episode):
    epsilon = 0.5 * (1 / (episode + 1)) #徐々に最適行動のみをとる、ε-greedy法
    if epsilon <= np.random.uniform(0, 1):
        next_action = np.argmax(q_table[next_state]) # 以上ならmax
    else:
        next_action = np.random.choice([0, 1]) # そうじゃなければrandom
    return next_action


# ---------- Qテーブルを更新する関数 --------------------------
def update_Qtable(q_table, state, action, reward, next_state):
    gamma = 0.99
    alpha = 0.5
    # ふーむ
    next_Max_Q=max(q_table[next_state][0],q_table[next_state][1] )
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * next_Max_Q)
   
    return q_table

In [4]:
# ---------- メイン関数開始 パラメータ設定-------------
env = gym.make('CartPole-v0')
max_number_of_steps = 200  #1試行のstep数
num_consecutive_iterations = 100  #学習完了評価に使用する平均試行回数
num_episodes = 2000  #総試行回数
goal_average_reward = 195  #この報酬を超えると学習終了（中心への制御なし）
# 状態を6分割^（4変数）にデジタル変換してQ関数（表）を作成
num_dizitized = 6  #分割数
q_table = np.random.uniform(
    low=-1, high=1, size=(num_dizitized**4, env.action_space.n))
 
total_reward_vec = np.zeros(num_consecutive_iterations)  #各試行の報酬を格納
final_x = np.zeros((num_episodes, 1))  #学習後、各試行のt=200でのｘの位置を格納
islearned = 0  #学習が終わったフラグ
isrender = 0  #描画フラグ

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [5]:
# [5] メインルーチン--------------------------------------------------
for episode in range(num_episodes):  #試行数分繰り返す
    # 環境の初期化
    observation = env.reset()
    state = digitize_state(observation)
    action = np.argmax(q_table[state])
    episode_reward = 0
 
    for t in range(max_number_of_steps):  #1試行のループ
        if islearned == 1:  #学習終了したらcartPoleを描画する
            env.render()
            time.sleep(0.1)
            print (observation[0])  #カートのx位置を出力
 
        # 行動a_tの実行により、s_{t+1}, r_{t}などを計算する
        observation, reward, done, info = env.step(action)
 
        # 報酬を設定し与える
        if done:
            if t < 195:
                reward = -200  #こけたら罰則
            else:
                reward = 1  #立ったまま終了時は罰則はなし
        else:
            reward = 1  #各ステップで立ってたら報酬追加
 
        episode_reward += reward  #報酬を追加
 
        # 離散状態s_{t+1}を求め、Q関数を更新する
        next_state = digitize_state(observation)  #t+1での観測状態を、離散値に変換
        q_table = update_Qtable(q_table, state, action, reward, next_state)
        
        #  次の行動a_{t+1}を求める 
        action = get_action(next_state, episode)    # a_{t+1} 
        
        state = next_state
        
        #終了時の処理
        if done:
            print('%d Episode finished after %f time steps / mean %f' %
                  (episode, t + 1, total_reward_vec.mean()))
            total_reward_vec = np.hstack((total_reward_vec[1:],
                                          episode_reward))  #報酬を記録
            if islearned == 1:  #学習終わってたら最終のx座標を格納
                final_x[episode, 0] = observation[0]
            break
 
    if (total_reward_vec.mean() >=
            goal_average_reward):  # 直近の100エピソードが規定報酬以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        islearned = 1
        #np.savetxt('learned_Q_table.csv',q_table, delimiter=",") #Qtableの保存する場合
        if isrender == 0:
            #env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
            isrender = 1
    #10エピソードだけでどんな挙動になるのか見たかったら、以下のコメントを外す
#     if episode>10:
#        if isrender == 0:
#            env = wrappers.Monitor(env, './movie/cartpole-experiment-1') #動画保存する場合
#            isrender = 1
#        islearned=1;
 
if islearned:
    np.savetxt('final_x.csv', final_x, delimiter=",")

0 Episode finished after 16.000000 time steps / mean 0.000000
1 Episode finished after 14.000000 time steps / mean -1.850000
2 Episode finished after 12.000000 time steps / mean -3.720000
3 Episode finished after 10.000000 time steps / mean -5.610000
4 Episode finished after 15.000000 time steps / mean -7.520000
5 Episode finished after 10.000000 time steps / mean -9.380000
6 Episode finished after 10.000000 time steps / mean -11.290000
7 Episode finished after 12.000000 time steps / mean -13.200000
8 Episode finished after 17.000000 time steps / mean -15.090000
9 Episode finished after 10.000000 time steps / mean -16.930000
10 Episode finished after 12.000000 time steps / mean -18.840000
11 Episode finished after 17.000000 time steps / mean -20.730000
12 Episode finished after 29.000000 time steps / mean -22.570000
13 Episode finished after 83.000000 time steps / mean -24.290000
14 Episode finished after 58.000000 time steps / mean -25.470000
15 Episode finished after 67.000000 time s

129 Episode finished after 200.000000 time steps / mean 6.000000
130 Episode finished after 200.000000 time steps / mean 9.700000
131 Episode finished after 200.000000 time steps / mean 11.820000
132 Episode finished after 200.000000 time steps / mean 11.820000
133 Episode finished after 200.000000 time steps / mean 11.820000
134 Episode finished after 89.000000 time steps / mean 13.960000
135 Episode finished after 200.000000 time steps / mean 13.010000
136 Episode finished after 200.000000 time steps / mean 15.880000
137 Episode finished after 200.000000 time steps / mean 19.460000
138 Episode finished after 200.000000 time steps / mean 22.450000
139 Episode finished after 185.000000 time steps / mean 26.240000
140 Episode finished after 200.000000 time steps / mean 27.670000
141 Episode finished after 200.000000 time steps / mean 30.380000
142 Episode finished after 183.000000 time steps / mean 30.380000
143 Episode finished after 200.000000 time steps / mean 30.660000
144 Episode f

254 Episode finished after 161.000000 time steps / mean 46.530000
255 Episode finished after 200.000000 time steps / mean 44.130000
256 Episode finished after 200.000000 time steps / mean 44.130000
257 Episode finished after 200.000000 time steps / mean 46.300000
258 Episode finished after 200.000000 time steps / mean 46.300000
259 Episode finished after 200.000000 time steps / mean 48.730000
260 Episode finished after 141.000000 time steps / mean 51.260000
261 Episode finished after 114.000000 time steps / mean 51.020000
262 Episode finished after 179.000000 time steps / mean 50.440000
263 Episode finished after 200.000000 time steps / mean 48.220000
264 Episode finished after 200.000000 time steps / mean 48.220000
265 Episode finished after 107.000000 time steps / mean 48.220000
266 Episode finished after 116.000000 time steps / mean 48.050000
267 Episode finished after 161.000000 time steps / mean 45.200000
268 Episode finished after 114.000000 time steps / mean 44.910000
269 Episod

385 Episode finished after 200.000000 time steps / mean 139.430000
386 Episode finished after 200.000000 time steps / mean 141.790000
387 Episode finished after 140.000000 time steps / mean 144.380000
388 Episode finished after 133.000000 time steps / mean 141.770000
389 Episode finished after 140.000000 time steps / mean 139.090000
390 Episode finished after 146.000000 time steps / mean 138.720000
391 Episode finished after 178.000000 time steps / mean 136.170000
392 Episode finished after 171.000000 time steps / mean 133.940000
393 Episode finished after 152.000000 time steps / mean 131.640000
394 Episode finished after 200.000000 time steps / mean 131.620000
395 Episode finished after 200.000000 time steps / mean 133.860000
396 Episode finished after 152.000000 time steps / mean 136.240000
397 Episode finished after 200.000000 time steps / mean 136.150000
398 Episode finished after 178.000000 time steps / mean 136.150000
399 Episode finished after 186.000000 time steps / mean 133.92

514 Episode finished after 200.000000 time steps / mean 130.210000
515 Episode finished after 200.000000 time steps / mean 130.210000
516 Episode finished after 200.000000 time steps / mean 130.210000
517 Episode finished after 200.000000 time steps / mean 130.210000
518 Episode finished after 200.000000 time steps / mean 130.210000
519 Episode finished after 200.000000 time steps / mean 132.420000
520 Episode finished after 200.000000 time steps / mean 132.420000
521 Episode finished after 200.000000 time steps / mean 135.010000
522 Episode finished after 200.000000 time steps / mean 135.010000
523 Episode finished after 200.000000 time steps / mean 137.240000
524 Episode finished after 200.000000 time steps / mean 137.240000
525 Episode finished after 200.000000 time steps / mean 137.240000
526 Episode finished after 200.000000 time steps / mean 137.240000
527 Episode finished after 200.000000 time steps / mean 137.240000
528 Episode finished after 200.000000 time steps / mean 137.24

KeyboardInterrupt: 